In [2]:
cd /content/drive/MyDrive/FinalProject/CvT-main

/content/drive/MyDrive/FinalProject/CvT-main


In [3]:
# Install the same pytorch as the paper
!pip install torch==1.7.1 torchvision==0.8.2 torchaudio==0.7.2 

# Install a environment with 'requirement.txt' attached to the paper
!python -m pip install -r requirements.txt --user -q 

# Required for code operation but omitting in the paper
!pip install -U PyYAML

     |████████████████████████████████| 776.8 MB 16 kB/s 
     |████████████████████████████████| 12.8 MB 50.8 MB/s 
     |████████████████████████████████| 7.6 MB 63.5 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.12.0+cu113
    Uninstalling torchvision-0.12.0+cu113:
      Successfully uninstalled torchvision-0.12.0+cu113
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.11.0+cu113
    Uninstalling torchaudio-0.11.0+cu113:
      Successfully uninstalled torchaudio-0.11.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.7.1 which is incompatible.
     |████

In [4]:
import sys

sys.path.append('/content/drive/MyDrive/FinalProject/CvT-main')
sys.path.append('/content/drive/MyDrive/FinalProject/CvT-main/lib')
sys.path.append('/content/drive/MyDrive/FinalProject/CvT-main/lib/tools')
sys.path.append('/root/.local/lib/python3.7/site-packages')

In [5]:
# Check dataset is kind of ImageNet
def build_dataset(cfg, is_train):
    dataset = None
    if 'imagenet' in cfg.DATASET.DATASET:
        dataset = _build_imagenet_dataset(cfg, is_train)
    else:
        raise ValueError('Unkown dataset: {}'.format(cfg.DATASET.DATASET))
    return dataset

def build_dataloader(cfg, is_train=True, distributed=False):
    if is_train:
        batch_size_per_gpu = cfg.TRAIN.BATCH_SIZE_PER_GPU
        shuffle = True
    else:
        batch_size_per_gpu = cfg.TEST.BATCH_SIZE_PER_GPU
        shuffle = False

    dataset = build_dataset(cfg, is_train)

    if distributed:
        if is_train and cfg.DATASET.SAMPLER == 'repeated_aug':
            logging.info('=> use repeated aug sampler')
            sampler = RASampler(dataset, shuffle=shuffle)
        else:
            sampler = torch.utils.data.distributed.DistributedSampler(
                dataset, shuffle=shuffle
            )
        shuffle = False
    else:
        sampler = None

    if cfg.AUG.TIMM_AUG.USE_LOADER and is_train:
        logging.info('=> use timm loader for training')
        timm_cfg = cfg.AUG.TIMM_AUG
        data_loader = create_loader(
            dataset,
            input_size=cfg.TRAIN.IMAGE_SIZE[0],
            batch_size=cfg.TRAIN.BATCH_SIZE_PER_GPU,
            is_training=True,
            use_prefetcher=True,
            no_aug=False,
            re_prob=timm_cfg.RE_PROB,
            re_mode=timm_cfg.RE_MODE,
            re_count=timm_cfg.RE_COUNT,
            re_split=timm_cfg.RE_SPLIT,
            scale=cfg.AUG.SCALE,
            ratio=cfg.AUG.RATIO,
            hflip=timm_cfg.HFLIP,
            vflip=timm_cfg.VFLIP,
            color_jitter=timm_cfg.COLOR_JITTER,
            auto_augment=timm_cfg.AUTO_AUGMENT,
            num_aug_splits=0,
            interpolation=timm_cfg.INTERPOLATION,
            mean=cfg.INPUT.MEAN,
            std=cfg.INPUT.STD,
            num_workers=cfg.WORKERS,
            distributed=distributed,
            collate_fn=None,
            pin_memory=cfg.PIN_MEMORY,
            use_multi_epochs_loader=True
        )
    else:
        data_loader = torch.utils.data.DataLoader(
            dataset,
            batch_size=batch_size_per_gpu,
            shuffle=shuffle,
            num_workers=cfg.WORKERS,
            pin_memory=cfg.PIN_MEMORY,
            sampler=sampler,
            drop_last=True if is_train else False,
        )

    return data_loader


In [6]:
!pip install ptflops

In [7]:
import os
import logging


from timm.data import create_loader
import torch
import torchvision
import torch.utils.data
import torchvision.datasets as datasets

from config import config
from config import update_config
from config import save_config
from dataset.transformas.build import build_transforms
from dataset.build import _build_imagenet_dataset
from dataset.transformas import build_transforms
from dataset.samplers import RASampler
from utils.utils import create_logger
from utils.utils import init_distributed
from utils.utils import setup_cudnn
from utils.comm import comm

# Substitute argparse
class Args:
  def __init__(self, cfg, local_rank, port, opts):
    self.cfg = cfg
    self.local_rank = local_rank
    self.port = port
    self.opts = opts

# cfg에 데이터셋 이름과 배치사이즈, 에포크 등을 정의한 config 파일 경로 할당
cfg = '/content/drive/MyDrive/FinalProject/CvT-main/experiments/imagenette2/cvt/cvt-13-224x224-method-a.yaml'

# Parameter
opts = ['MODEL.SPEC.POS_EMBED', '[False, False, False]']

# Argument Instance
args = Args(cfg, 0, 9000, opts)

# Set num_gpus, distributed and specify the gpu to use (local_rank)
init_distributed(args)

# Set cudnn
setup_cudnn(config)

# Update the arguments in the yaml file to the config
update_config(config, args)
final_output_dir = create_logger(config, args.cfg, 'test')

# Check if the distributed pachkage is available
if comm.is_main_process():
  output_config_path = os.path.join(final_output_dir, 'config.yaml')
  save_config(config, output_config_path)

=> merge config from /content/drive/MyDrive/FinalProject/CvT-main/experiments/imagenette2/cvt/cvt-13-224x224-method-a.yaml
=> creating OUTPUT ...
=> creating OUTPUT/imagenette2/cvt-13-224x224-method-a ...
=> setup logger ...


In [8]:
dataloader = build_dataloader(config)

2022-05-19 02:43:33,269:[P:74]:Rank[0/1] => use timm loader for training


In [9]:
config.MODEL.SPEC.POS_EMBED

[False, False, False]

In [10]:
i = 0
for (x, y) in dataloader:
  print(x, y)
  i = i + 1
  if i >= 2:
    break

tensor([[[[ 1.9407,  1.9407,  1.9064,  ...,  0.7248,  0.6906,  0.6563],
          [ 2.0263,  1.9920,  2.0263,  ...,  0.6563,  0.6563,  0.6392],
          [ 2.0605,  2.0948,  2.0263,  ...,  0.7591,  0.6906,  0.6563],
          ...,
          [ 2.2489,  2.2489,  2.2489,  ..., -0.1999, -0.1143, -0.1999],
          [ 2.2489,  2.2489,  2.2489,  ...,  1.7523,  1.5982,  1.5639],
          [ 2.2489,  2.2489,  2.2489,  ...,  2.2489,  2.2489,  2.2489]],

         [[ 2.4286,  2.4286,  2.4286,  ...,  2.4286,  2.4286,  2.4286],
          [ 2.4286,  2.4286,  2.4286,  ...,  2.4286,  2.4286,  2.4286],
          [ 2.4286,  2.4286,  2.4286,  ...,  2.4286,  2.4286,  2.4286],
          ...,
          [ 2.4286,  2.4286,  2.4286,  ..., -0.0749, -0.0224, -0.1099],
          [ 2.4286,  2.4286,  2.4286,  ...,  1.6933,  1.5007,  1.4482],
          [ 2.4286,  2.4286,  2.4286,  ...,  1.9734,  1.9209,  1.8508]],

         [[ 2.6400,  2.6400,  2.6400,  ...,  2.6400,  2.6400,  2.6400],
          [ 2.6400,  2.6400,  